# The task is divided on 2 part:
- ## compare how parameters like: *country*, *professional*, *education* (formal and non-formal), *IT skill* and *work experience* impact on salary, 
- ## predict future salary for people whose salary value in `'NaN'`.

### Because schema has big number of different kind of questions (some of them are not connected with IT) not every columns are needed to make a job done.

In [1]:
import pandas as pd
import numpy as np

### Open new CSV file as ***'df_answers'***  DataFrame

In [2]:
df_answers = pd.read_csv('survey_results_public_clean.csv', index_col = False)
df_answers.head()

,Professional,ProgramHobby,Country,University,EmploymentStatus,FormalEducation,MajorUndergrad,HomeRemote,CompanySize,CompanyType,YearsProgram,YearsCodedJob,CareerSatisfaction,JobSatisfaction,Gender,Race,DeveloperType,HaveWorkedLanguage,Currency,Salary
0,Student,"Yes, both",United States,No,"Not employed, and not looking for work",Secondary school,NaN,NaN,NaN,NaN,2 to 3 years,NaN,NaN,NaN,Male,White or of European descent,NaN,Swift,NaN,NaN
1,Student,"Yes, both",United Kingdom,"Yes, full-time",Employed part-time,Some college/university study without earning ...,Computer science or software engineering,"More than half, but not all, the time",20 to 99 employees,"Privately-held limited company, not in startup...",9 to 10 years,NaN,NaN,NaN,Male,White or of European descent,NaN,JavaScript; Python; Ruby; SQL,British pounds sterling (£),NaN
2,Professional developer,"Yes, both",United Kingdom,No,Employed full-time,Bachelor's degree,Computer science or software engineering,"Less than half the time, but at least one day ...","10,000 or more employees",Publicly-traded corporation,20 or more years,20 or more years,8.0,9.0,Male,White or of European descent,Other,Java; PHP; Python,British pounds sterling (£),113750.0
3,Professional non-developer who sometimes write...,"Yes, both",United States,No,Employed full-time,Doctoral degree,A non-computer-focused engineering discipline,"Less than half the time, but at least one day ...","10,000 or more employees",Non-profit/non-governmental organization or pr...,14 to 15 years,9 to 10 years,6.0,3.0,Male,White or of European descent,NaN,Matlab; Python; R; SQL,NaN,NaN
4,Professional developer,"Yes, I program as a hobby",Switzerland,No,Employed full-time,Master's degree,Computer science or software engineering,Never,10 to 19 employees,"Privately-held limited company, not in startup...",20 or more years,10 to 11 years,6.0,8.0,NaN,NaN,Mobile developer; Graphics programming; Deskto...,NaN,NaN,NaN


In [3]:
## Info about our new dataset
df_answers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51392 entries, 0 to 51391
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Professional        51392 non-null  object 
 1   ProgramHobby        51392 non-null  object 
 2   Country             51392 non-null  object 
 3   University          51392 non-null  object 
 4   EmploymentStatus    51392 non-null  object 
 5   FormalEducation     51392 non-null  object 
 6   MajorUndergrad      42841 non-null  object 
 7   HomeRemote          44008 non-null  object 
 8   CompanySize         38922 non-null  object 
 9   CompanyType         38823 non-null  object 
 10  YearsProgram        51145 non-null  object 
 11  YearsCodedJob       40890 non-null  object 
 12  CareerSatisfaction  42695 non-null  float64
 13  JobSatisfaction     40376 non-null  float64
 14  Gender              35047 non-null  object 
 15  Race                33033 non-null  object 
 16  Deve

***

### Let's check how many unique values are in every columns (exclude 'Salary')

### List of all unique values in columns

In [4]:
def Dataframe_unique_values(df):
    # Default values for calculations
    multiply, summary = 1, 0
    ## Exclude 'Salary' from DataFrame
    new_df = df.drop('Salary', axis=1)
    for val in new_df.columns:
        unique_num = len(new_df[val].unique())
        print(f"Number of unique values in '{val}': {unique_num}")
        multiply *= unique_num
        summary += unique_num
    print('\n')
    print(f'Sum of all unique values in our columns are: {summary}')
    print(f'Number of all possible unique values combinations in dataset are: {multiply} and it is more than 10**{np.log10(float(multiply)):.0f}')
    return summary, multiply

unique_values_before = Dataframe_unique_values(df_answers)

Number of unique values in 'Professional': 5
Number of unique values in 'ProgramHobby': 4
Number of unique values in 'Country': 201
Number of unique values in 'University': 4
Number of unique values in 'EmploymentStatus': 7
Number of unique values in 'FormalEducation': 9
Number of unique values in 'MajorUndergrad': 17
Number of unique values in 'HomeRemote': 8
Number of unique values in 'CompanySize': 11
Number of unique values in 'CompanyType': 12
Number of unique values in 'YearsProgram': 22
Number of unique values in 'YearsCodedJob': 22
Number of unique values in 'CareerSatisfaction': 12
Number of unique values in 'JobSatisfaction': 12
Number of unique values in 'Gender': 30
Number of unique values in 'Race': 98
Number of unique values in 'DeveloperType': 1824
Number of unique values in 'HaveWorkedLanguage': 8439
Number of unique values in 'Currency': 18


Sum of all unique values in our columns are: 10755
Number of all possible unique values combinations in dataset are: 10324830806

## We have got a big mess, so we must reduce the number of unique values as far as we can - in the order show by the column names.

### Columns will be checking in the order below:
1) 'Country'
2) 'Professional', 'ProgramHobby', 'University', 'EmploymentStatus', 'FormalEducation'
3) 'MajorUndergrad', 'HomeRemote'
4) 'CompanySize', 'CompanyType'
5) 'YearsProgram', 'YearsCodedJob'
6) 'CareerSatisfaction', 'JobSatisfaction'
7) 'Gender'
8) 'Race'
9) 'DeveloperType'
10) 'HaveWorkedLanguage'
11) 'Currency'

### List of unique values in columns

In [5]:
def Unique_values(col):
    return df_answers[col].unique()

def Unique_values_list(*cols):
    limit = 25 ## Some columns have too many unique values to print them on the screen at once
    cols_list = list(cols)
    
    for _ in cols_list:
        col_unique_names = df_answers[_].value_counts(dropna=False)
        unique_numbers = len(col_unique_names)
    
        if unique_numbers > limit:
            print(f"Number of unique values in '{_}': {unique_numbers}\
            \nFirst {limit} unique values:\n{col_unique_names[:limit]}\n\n")
        else:
            print(f"Number of unique values in '{_}': {unique_numbers}\
            \nAll unique values:\n{col_unique_names}\n\n")

#### **1) 'Country'**

In [6]:
Unique_values_list('Country')

Number of unique values in 'Country': 201            
First 25 unique values:
United States         11455
India                  5197
United Kingdom         4395
Germany                4143
Canada                 2233
France                 1740
Poland                 1290
Australia               913
Russian Federation      873
Spain                   864
Netherlands             855
Italy                   781
Brazil                  777
Sweden                  611
Switzerland             595
Israel                  575
Romania                 561
Iran                    507
Austria                 477
Pakistan                454
Czech Republic          411
Belgium                 404
South Africa            380
Turkey                  363
Ukraine                 356
Name: Country, dtype: int64




#### Because many world countries' in this Series are probably the minority (small number of responders),
#### we will take countries with at least 50 representants - that is the simplest method of detection and removing outliers.

In [7]:
countries_names = df_answers.groupby('Country').filter(lambda x: len(x) >= 50)['Country'].unique()

print(f"Number of the filtered countries: {len(countries_names)}\n")
print(f"Names of the filtered countries: \n{countries_names}")

Number of the filtered countries: 81

Names of the filtered countries: 
['United States' 'United Kingdom' 'Switzerland' 'New Zealand' 'Poland'
 'Colombia' 'France' 'Canada' 'Germany' 'Greece' 'Brazil' 'Israel' 'Italy'
 'Belgium' 'India' 'Chile' 'Croatia' 'Argentina' 'Netherlands' 'Denmark'
 'Ukraine' 'Sri Lanka' 'Malaysia' 'Finland' 'Turkey' 'Spain' 'Austria'
 'Mexico' 'Russian Federation' 'Bulgaria' 'Uruguay' 'Estonia' 'Iran'
 'Bangladesh' 'Sweden' 'Lithuania' 'Romania' 'Costa Rica' 'Serbia'
 'Slovenia' 'United Arab Emirates' 'Tunisia' 'Kenya' 'Norway'
 'Dominican Republic' 'Belarus' 'Portugal' 'Czech Republic' 'Albania'
 'I prefer not to say' 'South Africa' 'Moldavia' 'Ireland' 'Nepal'
 'Pakistan' 'Slovak Republic' 'Hungary' 'Egypt' 'Australia' 'Japan'
 'South Korea' 'Vietnam' 'Saudi Arabia' 'Macedonia' 'Bosnia-Herzegovina'
 'Indonesia' 'Nigeria' 'Peru' 'Morocco' 'Armenia' 'Lebanon' 'China'
 'Latvia' 'Singapore' 'Thailand' 'Philippines' 'Hong Kong' 'Taiwan'
 'Afghanistan' 'Ghana' 'Ve

#### After making simple filtering we limit our countries to 81 positions (it is only 40% of total),
#### but we can also remove 'I prefer not to say' value which is useless and that will be made in the next step.

In [8]:
df_answers = df_answers[(df_answers['Country'].isin(countries_names)) & 
                        (~df_answers['Country'].isin(['I prefer not to say']))].reset_index(drop=True)
df_answers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50048 entries, 0 to 50047
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Professional        50048 non-null  object 
 1   ProgramHobby        50048 non-null  object 
 2   Country             50048 non-null  object 
 3   University          50048 non-null  object 
 4   EmploymentStatus    50048 non-null  object 
 5   FormalEducation     50048 non-null  object 
 6   MajorUndergrad      41807 non-null  object 
 7   HomeRemote          43004 non-null  object 
 8   CompanySize         38109 non-null  object 
 9   CompanyType         38015 non-null  object 
 10  YearsProgram        49825 non-null  object 
 11  YearsCodedJob       39961 non-null  object 
 12  CareerSatisfaction  41691 non-null  float64
 13  JobSatisfaction     39500 non-null  float64
 14  Gender              34324 non-null  object 
 15  Race                32380 non-null  object 
 16  Deve